<h2> Import Libraries </h2>

In [1]:
!pip install lime
!pip install shap
!pip install eli5

     |████████████████████████████████| 276kB 3.4MB/s 
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/8e/c1/c1cddd9cf8fbae812904fa5c84ef571e782891288d309d04c8
Successfully built lime
     |████████████████████████████████| 235kB 3.4MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
  Stored in directory: /home/jovyan/.cache/pip/wheels/14/42/0c/c07f5fdfb68e06b525d2ba984f50c83f1e5ebd3d2680852aa5
Successfully built shap
     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
  Stored in directory: /home/jovyan/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built tabulate


In [21]:
#Data handling
import pandas as pd
import numpy as np
import scipy as sp
import gc
import pickle
#preprocessing and feature selection
import sklearn.preprocessing
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.model_selection import train_test_split, RandomizedSearchCV
#models
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
#evaluation and interpretability
import lime
import shap
import eli5
import sklearn.metrics

<h2> Pre-process Data for Model Training </h2>

<b> STEP 1: Encode Datasets <b>

In [43]:
alldata = pd.read_pickle('sp500finaldata.pkl')
#The only features that need to be label encoded are 'Sector' and 'DistanceFromLast'
alldata.dtypes[alldata.dtypes=='object']

firm                object
Sector              object
DistanceFromLast    object
dtype: object

In [44]:
labenc = LabelEncoder()
labenc.fit(alldata['Sector'])
with open('labencsector.pkl', 'wb') as q:
    pickle.dump(labenc, q)
print(labenc.transform(alldata['Sector']))
alldata['Sector'] = labenc.transform(alldata['Sector'])
labenc1 = LabelEncoder()
labenc1.fit(alldata['DistanceFromLast'])
with open('labencdist.pkl', 'wb') as q:
    pickle.dump(labenc1, q)
print(labenc1.transform(alldata['DistanceFromLast']))
alldata['DistanceFromLast'] = labenc1.transform(alldata['DistanceFromLast'])


[4 4 4 ... 4 4 4]
[6 0 0 ... 0 0 0]


In [45]:
onehot = OneHotEncoder(sparse=False)
#Since we would be dropping the 'firm' feature, only 'Sector' and 'DistanceFromLast' from above need to be hot-encoded
onehot.fit(alldata[['Sector', 'DistanceFromLast']])
with open('onehotenc.pkl', 'wb') as q:
    pickle.dump(onehot, q)
print(onehot.transform(alldata[['Sector', 'DistanceFromLast']]))
namessector = ['Sector_'+str(i) for i in labenc.classes_]
namesdist = ['DistLast_'+str(i) for i in labenc1.classes_]
names = np.append(namessector, namesdist)
enc = onehot.transform(alldata[['Sector', 'DistanceFromLast']])
enc = pd.DataFrame(enc, columns=names)
alldata.drop(['firm', 'Sector', 'DistanceFromLast'], axis=1, inplace=True)
alldata = pd.concat([alldata, enc], axis=1)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


<b> STEP 2 - Separate Two Models </b>

We divide our project into two distinct models - one where we attempt to predict 'Div_Paid?' (hence a classifier model predicting whether or not dividends will be paid), and another where we attempt to predict how much dividend, if paid (therefore a regressor model). Let us designate the different datasets for both.

In [55]:
classifdata = alldata.copy()
classifdata.drop(['Dividend'], axis=1, inplace=True)
regrdata = alldata[alldata['Div_Paid?']==1].copy()
regrdata.drop(['Div_Paid?'], axis=1, inplace=True)
classifdata.to_pickle('classifdatafull.pkl')
regrdata.to_pickle('regrdatafull.pkl')

<h2> Classifier Model - Dividend Outlook </h2>

 First we work towards the classifier model, and begin with feature selection. Subsequently, we try three different fits - RandomForestClassifier, GaussianProcessClassifier and MLPClassifier.

<b> STEP 1 - Feature Elimination </b>

We perform basic feature elimination here, and leave actual selection to fit based on model. The only selection performed here is chi2 testing (to check relationship to the target variable) and VarianceThreshold (differences within the feature).

In [46]:
t = VarianceThreshold()
t.fit(alldata[''])

VarianceThreshold(threshold=0.0)

In [52]:
alldata.columns.values[t.get_support(indices=False)==False]

array(['Accumulated Depreciation'], dtype=object)